We want to send and Receive Data from a process

1. Create a new process that launches the command line argument
2. Send the string Testing\n to that process
3. Receive any data it writes to standard output

## A More Convenient API – `execlp`

    int execlp(const char *file, const char *arg /⪗ Ⳬⴷⶃ, (char *) NULL ⁎/);

Does not return on success, and -1 on failure (and sets errno)  

`execlp` will let you skip using string arrays (using C varargs), and it will also search for executables using the PATH environment variable

### Our Final APIs — `dup` and `dup2`

    int dup(int oldfd);
    
    int dup2(int oldfd, int newfd);

Returns a new file descriptor on success, and -1 on failure (and sets errno)

Copies the file descriptor so oldfd and newfd refer to the same thing

For `dup` it’ll return the lowest file descriptor

For `dup2` it’ll atomically close the newfd argument (if open), and then make newfd refer to the same thing